In [1]:
import torch

In [7]:
torch.tensor([[3.,4.],
              [2,4]]).shape

torch.Size([2, 2])

In [9]:
x =  torch.tensor(3.)
w = torch.tensor(4., requires_grad=True)
b = torch.tensor(2., requires_grad=True)

In [10]:
y = w * x + b
y

tensor(14., grad_fn=<AddBackward0>)

In [11]:
y.backward()

In [15]:
print(w.grad)

tensor(3.)


In [2]:
t1 = torch.full((3,2),10)

In [3]:
t1

tensor([[10, 10],
        [10, 10],
        [10, 10]])

In [4]:
import numpy as np

In [8]:
x = np.array([[1,2],[2,3.]])
k = torch.from_numpy(x)

In [9]:
k

tensor([[1., 2.],
        [2., 3.]], dtype=torch.float64)

## <span style='color:red;'> Gradient Descent and Linear Regression </span>

In [10]:
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [11]:
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [12]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [13]:
w = torch.randn(2,3,requires_grad=True)
b = torch.randn(2, requires_grad=True)

In [26]:
w

tensor([[-0.7630,  0.0659, -0.1013],
        [ 0.6760, -0.1359,  0.5560]], requires_grad=True)

In [15]:
# @ represents matrix multiplications and .t represents transpose
def model(x):
    return x @ w.t() + b

In [16]:
preds = model(inputs)
print(preds)

tensor([[-54.8584,  64.7160],
        [-69.3347,  85.7067],
        [-62.6419,  73.4152],
        [-77.9589,  84.2455],
        [-52.6299,  73.0836]], grad_fn=<AddBackward0>)


In [17]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [18]:
def mse(t1,t2):
    diff = t1-t2
    return torch.sum(diff*diff)/diff.numel()

In [20]:
loss = mse(preds,targets)
loss

tensor(11024.8975, grad_fn=<DivBackward0>)

In [21]:
#Computing gradients

In [23]:
loss.backward()

In [24]:
print(w)

tensor([[-0.7630,  0.0659, -0.1013],
        [ 0.6760, -0.1359,  0.5560]], requires_grad=True)


In [25]:
print(w.grad)

tensor([[-11702.0498, -12847.1377,  -7903.2285],
        [ -1062.0981,  -2412.1238,  -1225.5934]])


In [28]:
print(b)
print(b.grad)

tensor([0.7766, 0.5640], requires_grad=True)
tensor([-139.6848,  -15.7666])


In [29]:
#Why?
# We are going to use them toadjust weights and biases to reduce the loss
# algebra

In [35]:
with torch.no_grad():
    w-=w.grad * 1e-5
    b-=b.grad * 1e-5

In [31]:
# We use torch.no_grad to indicate PyTrch that we shouldn'y track and calculate or modelify gradients while
# updating the weights and biases

In [38]:
loss = mse(model(inputs),targets)
print(loss)

tensor(4899.5669, grad_fn=<DivBackward0>)


In [39]:
w.grad.zero_()
b.grad.zero_()

tensor([0., 0.])

In [40]:
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


## <span style='color:red;'> Gradient Descent </span>

### <span style='color:red;'> Step 1: Generate Predictions </span>

In [44]:
preds = model(inputs)
print(preds)

tensor([[-13.7587,  70.5532],
        [-15.3071,  93.4541],
        [  1.3206,  83.1498],
        [-37.1870,  89.3939],
        [ -0.7473,  80.8967]], grad_fn=<AddBackward0>)


### <span style='color:red;'> Step 2: Calculate Loss </span>

In [45]:
loss = mse(preds, targets)
print(loss)

tensor(4899.5669, grad_fn=<DivBackward0>)


### <span style='color:red;'> Step 3: Calculate the gradient </span>

In [46]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-7458.0171, -8284.5381, -5088.1836],
        [ -453.6426, -1742.3765,  -815.8239]])
tensor([-89.3359,  -8.5105])


### <span style='color:red;'> Step 4: Update Weights and reset the gradients </span>

In [50]:
with torch.no_grad():
    w-=w.grad*1e-5
    b-=b.grad*1e-5
    w.grad.zero_()
    b.grad.zero_()

### <span style='color:red;'> Step 5: Look at the updated weights and biases and new loss  </span>

In [77]:
print(w)
print(b)
print(mse(model(inputs),targets))

tensor([[-0.2049,  0.5814,  0.2313],
        [ 0.6358, -0.0160,  0.5449]], requires_grad=True)
tensor([0.6943, 0.5016], requires_grad=True)
tensor(1169.6094, grad_fn=<DivBackward0>)


## <span style='color:Blue;'> Combining All for 100 epochs </span>

In [75]:
for i in range(0,100):
    preds = model(inputs)
    loss = mse(preds,targets)
    loss.backward()
    with torch.no_grad():
        w-=w*1e-5
        b-=b*1e-5
        w.grad.zero_()
        b.grad.zero_()


In [76]:
loss = mse(model(inputs),targets)
loss

tensor(1169.6094, grad_fn=<DivBackward0>)

## <span style='color:Blue;'> Linear Regression </span>

In [80]:
import torch.nn as nn

In [78]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

In [79]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

In [81]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

## <span style='color:Blue;'> Dataset and Data Loader </span>

In [83]:
from torch.utils.data import TensorDataset

In [86]:
train_ds = TensorDataset(inputs,targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [87]:
from torch.utils.data import DataLoader

In [88]:
batch_size = 5
train_dl = DataLoader(train_ds,batch_size,shuffle=True)

In [90]:
for xb,yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 73.,  66.,  44.],
        [ 88., 134.,  59.],
        [ 73.,  67.,  43.],
        [ 92.,  87.,  64.],
        [ 91.,  88.,  64.]])
tensor([[ 57.,  69.],
        [118., 132.],
        [ 56.,  70.],
        [ 82., 100.],
        [ 81., 101.]])


In [91]:
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.5011,  0.0952, -0.4953],
        [ 0.4602,  0.5239, -0.2693]], requires_grad=True)
Parameter containing:
tensor([-0.5747,  0.2738], requires_grad=True)


In [92]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.5011,  0.0952, -0.4953],
         [ 0.4602,  0.5239, -0.2693]], requires_grad=True),
 Parameter containing:
 tensor([-0.5747,  0.2738], requires_grad=True)]

In [93]:
model(inputs)

tensor([[21.0843, 57.3935],
        [21.7017, 71.0250],
        [27.0477, 94.8979],
        [36.3026, 59.7832],
        [ 8.4679, 63.4751],
        [21.4902, 57.3299],
        [21.1112, 70.2319],
        [27.0535, 95.0889],
        [35.8967, 59.8469],
        [ 7.4715, 62.7456],
        [20.4938, 56.6004],
        [22.1076, 70.9614],
        [27.6382, 95.6910],
        [37.2989, 60.5127],
        [ 8.0620, 63.5387]], grad_fn=<AddmmBackward0>)

## <span style='color:Blue;'> Loss Function </span>

In [94]:
import torch.nn.functional as F

In [95]:
loss_fn = F.mse_loss
loss = loss_fn(model(inputs),targets)
print(loss)

tensor(2846.2886, grad_fn=<MseLossBackward0>)


## <span style='color:Blue;'> Optimizer </span>

In [96]:
opt = torch.optim.SGD(model.parameters(),lr=1e-5)

## <span style='color:Blue;'> Model Training </span>

In [102]:
def fit(num_epochs,model,loss_fn,opt,train_dl):
    for epoch in range(num_epochs):
        
        # train which batches of data
        for xb,yb in train_dl:
            pred = model(xb)
            loss = loss_fn(pred,yb)
            loss.backward()
            #update parameters using gradients
            opt.step()
            opt.zero_grad()
        if (epoch+1)%10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

In [103]:
fit(100,model,loss_fn,opt,train_dl)

Epoch [10/100], Loss: 334.8664
Epoch [20/100], Loss: 445.7016
Epoch [30/100], Loss: 392.7287
Epoch [40/100], Loss: 185.6640
Epoch [50/100], Loss: 149.9113
Epoch [60/100], Loss: 183.4355
Epoch [70/100], Loss: 85.0648
Epoch [80/100], Loss: 98.0049
Epoch [90/100], Loss: 57.8164
Epoch [100/100], Loss: 110.8402


## <span style='color:Blue;'> Difference Between Classical ProG and ML </span>

Classical Programming:
Rules + Data -> Answers

Machine Learning :
Data + Answers -> Rules

## <span style='color:Green;'> ------------------------------------------------------------------------------------ </span>

## <span style='color:Blue;'> Assignment: Write about 5 pytorch functions </span>

### <span style='color:Blue;'> Torch.linspace : used to create 1d equally spaced tensor between the values start and end </span>

In [113]:
torch.linspace(1,10,100)

tensor([ 1.0000,  1.0909,  1.1818,  1.2727,  1.3636,  1.4545,  1.5455,  1.6364,
         1.7273,  1.8182,  1.9091,  2.0000,  2.0909,  2.1818,  2.2727,  2.3636,
         2.4545,  2.5455,  2.6364,  2.7273,  2.8182,  2.9091,  3.0000,  3.0909,
         3.1818,  3.2727,  3.3636,  3.4545,  3.5455,  3.6364,  3.7273,  3.8182,
         3.9091,  4.0000,  4.0909,  4.1818,  4.2727,  4.3636,  4.4545,  4.5455,
         4.6364,  4.7273,  4.8182,  4.9091,  5.0000,  5.0909,  5.1818,  5.2727,
         5.3636,  5.4545,  5.5455,  5.6364,  5.7273,  5.8182,  5.9091,  6.0000,
         6.0909,  6.1818,  6.2727,  6.3636,  6.4545,  6.5455,  6.6364,  6.7273,
         6.8182,  6.9091,  7.0000,  7.0909,  7.1818,  7.2727,  7.3636,  7.4545,
         7.5455,  7.6364,  7.7273,  7.8182,  7.9091,  8.0000,  8.0909,  8.1818,
         8.2727,  8.3636,  8.4545,  8.5455,  8.6364,  8.7273,  8.8182,  8.9091,
         9.0000,  9.0909,  9.1818,  9.2727,  9.3636,  9.4545,  9.5455,  9.6364,
         9.7273,  9.8182,  9.9091, 10.00

### <span style='color:Blue;'> torch.eye : creates 2d tensor with diagnols as 1 and others as 0</span>

In [117]:
torch.eye(5,5)

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])

In [119]:
#In case of non symmetrical dimesnions
torch.eye(3,2)

tensor([[1., 0.],
        [0., 1.],
        [0., 0.]])

### <span style='color:Blue;'> torch.unbind: removes a tensor dimension along a given dimension </span>

In [124]:
t = torch.tensor([[10,20,30,40,50],[1,2,3,4,5]])
torch.unbind(t)

(tensor([10, 20, 30, 40, 50]), tensor([1, 2, 3, 4, 5]))

In [125]:
t = torch.tensor([[10,20,30,40,50],[1,2,3,4,5]])
torch.unbind(t,axis=1)

(tensor([10,  1]),
 tensor([20,  2]),
 tensor([30,  3]),
 tensor([40,  4]),
 tensor([50,  5]))

### <span style='color:Blue;'> torch.get_num_threads() : Returns the number of threads used for parallelizing CPU operations </span>

In [127]:
torch.get_num_threads()

8

### <span style='color:Blue;'> torch.expand(*size) : set qty of elements per dimension</span>

In [135]:
x = torch.tensor([[1],[2],[3]])

In [136]:
print(x.size())

torch.Size([3, 1])


In [142]:
print(x.expand(3,5))

tensor([[1, 1, 1, 1, 1],
        [2, 2, 2, 2, 2],
        [3, 3, 3, 3, 3]])


In [145]:
#-1 means not cahnging the dimension of its size
print(x.expand(-1,3))

tensor([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3]])


In [151]:
a = torch.tensor([1])
display(a.size())
b = torch.tensor([1,2,3])
display(b.size())
c = torch.tensor([[1],[2],[3]])
display(c.size())

torch.Size([1])

torch.Size([3])

torch.Size([3, 1])

### <span style='color:Blue;'> torch.numel() : Returns the number of elements in input tensor </span>

In [156]:
t = torch.tensor([[1],[2],[3]])
display(t.size())
display(t.numel())
t = t.expand(-1,4)
display(t.size())
display(t.numel())

torch.Size([3, 1])

3

torch.Size([3, 4])

12